# Spectral Filtering: 1D Butterworth

Filter with maximally flat passband. Designed for analog circuit implementation in 1930. Other filters produce faster roll-off but introduce ripple in the pass and/or stop-band.

Low-pass filter given by
$H_{LP}(u) = \dfrac{1}{\sqrt{1 + \left( \frac{u}{u_0} \right)^{2n}}}$
where $u_0$ denotes cut-off frequency (-3dB), and order $n$ controls slope of roll-off.

High-pass filter can be obtained from low-pass filter: $H^2_{HP}(u) = 1-H^2_{LP}(u)$. Band-pass filter can be obtained from cascaded low and high-pass filters: $H_{BP}(u) = H_{LP}(u) H_{HP}(u)$.
Band-stop filter can be obtained from band-pass filter: $H^2_{BS}(u) = 1-H^2_{BP}(u)$.

In [ ]:
%matplotlib inline

import numpy as np

import matplotlib.pyplot as plt

from scipy import signal
from scipy.fft import fft, ifft, fftshift

In [ ]:
def plot(f, titles=None, show_kernel=False):
    fig, ax = plt.subplots(1, len(f), figsize=(12,3))
    
    for i in np.arange(0, len(f)):
        N = len(f[i])
        
        if show_kernel == False:
            ax[i].plot(f[i])
            ax[i].set_xticks(np.arange(0,N+1,N/8))
        else:
            K = N//2
            ax[i].plot(np.arange(-K,K+1),f[i])
            ax[i].set_xticks(np.arange(-K,K+1,K/4))
            
        ax[i].grid()
        if titles != None:
            ax[i].set_title(titles[i])               
    
    fig.tight_layout()

In [ ]:
def bwLP(u0, n, N):  
    u = np.arange(-N//2, N//2)
    H = 1/np.sqrt(1+(u/u0)**(2*n))
    return fftshift(H)

def bwHP(u0, n, N):
    LP = bwLP(u0, n, N)
    return np.sqrt(1-LP**2)

def bwBP(u1, n1, u2, n2, N):
    LP = bwLP(u2, n2, N)
    HP = bwHP(u1, n1, N)
    return LP*HP

def bwBS(u1, n1, u2, n2, N):
    BP = bwBP(u1, n1, u2, n2, N)
    return np.sqrt(1-BP**2)

In [ ]:
N = 256

(u0, n0) = (40, 8)
(u1, n1) = (16, 12)
(u2, n2) = (64, 12)

LP = bwLP(u0, n0, N)
HP = bwHP(u0, n0, N)
BP = bwBP(u1, n1, u2, n2, N)
BS = bwBS(u1, n1, u2, n2, N)

plot([np.real(LP), np.real(HP)], ['Low-Pass','High-Pass'])
plot([np.real(BP), np.real(BS)], ['Band-Pass','Band-Stop'])

## Example 1: Separating Signals

In [ ]:
T = 2.0*np.pi

x = np.arange(0, N)
f = np.sin(4*T*x/N) + 0.5*np.sin(32*T*x/N)

F = fft(f)

G1 = F*bwLP(18, 8, N)
g1 = np.real(ifft(G1))

G2 = F*bwHP(18, 8, N)
g2 = np.real(ifft(G2))

plot([f, np.abs(F)], ['Signal','FFT: |F|)'])
plot([g1, g2], ['LP Result','HP Result'])

## Example 2: Suppressing Noise

In [ ]:
f = np.sin(8*T*x/N)
n = 0.25*np.random.standard_normal(N)
f += n - np.sum(n)/N # make zero mean

F = fft(f)
H = bwLP(16, 8, N)

G = F*H
g = np.real(ifft(G))

plot([f, g], ['Signal','LP Result'])
plot([np.abs(F), np.abs(G)], ['FFT: |F|','FFT: |FH|'])

## Example 3: Convolution Kernels

In [ ]:
K = 10
M = N//2

h_low = fftshift(np.real(ifft(LP)))[M-K:M+K+1]    # sum == ?
h_high = fftshift(np.real(ifft(HP)))[M-K:M+K+1]   # sum == ?
h_pass = fftshift(np.real(ifft(BP)))[M-K:M+K+1]   # sum == ?
h_stop = fftshift(np.real(ifft(BS)))[M-K:M+K+1]   # sum == ?


plot([h_low, h_high],['Low-Pass','High-Pass'], show_kernel=True)
plot([h_pass, h_stop],['Band-Pass','Band-Stop'], show_kernel=True)